# Can be ignored

# **Modelling**

In [ ]:
!pip install -q transformers datasets scikit-learn pandas

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Final_Tweet_Data.csv to Final_Tweet_Data.csv


In [ ]:
import pandas as pd
import re

# Load CSV
df = pd.read_csv('/content/Final_Tweet_Data.csv')

# Drop missing values
df = df.dropna(subset=['text', 'Sentiment Score'])

# Clean text
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_text'] = df['text'].apply(clean_tweet)

# Check sentiment class balance
print(df['Sentiment Score'].value_counts())
df.head()

Sentiment Score
-1.0     3054
 0.0     2139
 1.0     2041
-11.0       1
Name: count, dtype: int64


,Brand,Factor,id,url,text,Sentiment Score,Tweet Type,clean_text
0,Citroen,Price,1924160628727447785,https://x.com/TorqueIndia/status/1924160628727...,"""@MenonNinad Most of the cars comes with simil...",1.0,Consumer,Most of the cars comes with similar features a...
1,Citroen,Price,1919053455513235793,https://x.com/grok/status/1919053455513235793,"""The Leapmotor T03 hatchback is expected to la...",1.0,Consumer,The Leapmotor T03 hatchback is expected to lau...
2,Citroen,Price,1919046080164958454,https://x.com/Xroaders_001/status/191904608016...,"""📣 Stellantis-Group a lease of new life or Las...",1.0,Consumer,StellantisGroup a lease of new life or Last op...
3,Citroen,Price,1850832200490426383,https://x.com/RBlyskye7/status/185083220049042...,"""But unfortunately Citroën and Renault in Indo...",1.0,Consumer,But unfortunately Citron and Renault in Indone...
4,Citroen,Price,1830534864191271150,https://x.com/TorqueIndia/status/1830534864191...,"""Citroen BASALT turbo affordable variants laun...",0.0,Consumer,Citroen BASALT turbo affordable variants launc...


As you can see above there was a value/Sentiment Score of -11, which we droped in the next code chunk

In [ ]:
import pandas as pd
import re

# Load CSV
df = pd.read_csv('/content/Final_Tweet_Data.csv')

# Drop rows with missing text or sentiment
df = df.dropna(subset=['text', 'Sentiment Score'])

# Convert sentiment to integer and drop invalid (-11)
df['Sentiment Score'] = df['Sentiment Score'].astype(int)
df = df[df['Sentiment Score'].isin([-1, 0, 1])]

# Clean tweet text
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r'\@\w+|\#','', text)                  # remove @mentions and hashtags
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)           # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()             # normalize whitespace
    return text

df['clean_text'] = df['text'].apply(clean_tweet)

# Check class distribution
print(df['Sentiment Score'].value_counts())
df.head()

Sentiment Score
-1    3054
 0    2139
 1    2041
Name: count, dtype: int64


,Brand,Factor,id,url,text,Sentiment Score,Tweet Type,clean_text
0,Citroen,Price,1924160628727447785,https://x.com/TorqueIndia/status/1924160628727...,"""@MenonNinad Most of the cars comes with simil...",1,Consumer,Most of the cars comes with similar features a...
1,Citroen,Price,1919053455513235793,https://x.com/grok/status/1919053455513235793,"""The Leapmotor T03 hatchback is expected to la...",1,Consumer,The Leapmotor T03 hatchback is expected to lau...
2,Citroen,Price,1919046080164958454,https://x.com/Xroaders_001/status/191904608016...,"""📣 Stellantis-Group a lease of new life or Las...",1,Consumer,StellantisGroup a lease of new life or Last op...
3,Citroen,Price,1850832200490426383,https://x.com/RBlyskye7/status/185083220049042...,"""But unfortunately Citroën and Renault in Indo...",1,Consumer,But unfortunately Citron and Renault in Indone...
4,Citroen,Price,1830534864191271150,https://x.com/TorqueIndia/status/1830534864191...,"""Citroen BASALT turbo affordable variants laun...",0,Consumer,Citroen BASALT turbo affordable variants launc...


# Text Classification Model Using RoBERTa

In [ ]:
!pip install -q transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

In [ ]:
# Rename for HuggingFace compatibility
df_model = df[['clean_text', 'Sentiment Score']].rename(columns={
    'clean_text': 'text',
    'Sentiment Score': 'label'
})

# HuggingFace expects labels as integers
df_model['label'] = df_model['label'].astype(int)

# Split into train/test
train_df, test_df = train_test_split(df_model, test_size=0.2, random_state=42, stratify=df_model['label'])

# Convert to HuggingFace datasets
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/5787 [00:00<?, ? examples/s]

Map:   0%|          | 0/1447 [00:00<?, ? examples/s]

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    weight_decay=0.01
)

In [ ]:
!pip install --upgrade --force-reinstall transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Step 1: Re-import necessary libraries
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [ ]:
# Load your cleaned data again
df = pd.read_csv('/content/Final_Tweet_Data.csv')

# Drop any invalid or NaN Sentiment Score entries (e.g., -11 or NaN)
df = df[df['Sentiment Score'].isin([-1, 0, 1])].dropna(subset=['text', 'Sentiment Score'])

# Clean tweets
import re

def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_text'] = df['text'].apply(clean_tweet)
df['label'] = df['Sentiment Score'].astype(int)

# Load tokenizer
from transformers import AutoTokenizer

model_name = "bert-base-uncased"  # You can switch to "roberta-base" too
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize
def tokenize(batch):
    return tokenizer(batch['clean_text'], padding=True, truncation=True)

# Convert to HuggingFace dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df[['clean_text', 'label']])
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Split
train_ds, test_ds = dataset.train_test_split(test_size=0.2).values()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/7234 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load pre-trained model for 3-class classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)

# 🚀 Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none"  # 👈 disables wandb and other logging integrations
)

ValueError: --load_best_model_at_end requires the save and eval strategy to match, but found
- Evaluation strategy: IntervalStrategy.NO
- Save strategy: SaveStrategy.EPOCH

In [ ]:
load_best_model_at_end=True

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",         # ✅ Add this
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none"                     # ✅ Disable wandb
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=True,                    # ✅ enable evaluation
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,                  # ✅ save every 500 steps
    eval_steps=500,                  # ✅ evaluate every 500 steps
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=False,   # ❌ not supported without matching strategies
    report_to=None                  # ✅ disable wandb
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
report_to=None  # ✅ disables W&B


In [ ]:
report_to=Nonetrainer.train()

NameError: name 'Nonetrainer' is not defined

# Model

In [ ]:
df = pd.read_csv("/content/Final_Tweet_Data.csv")

NameError: name 'pd' is not defined

In [ ]:
# Step 1: Drop any rows with missing or infinite values in 'Sentiment Score'
df = df[pd.to_numeric(df['Sentiment Score'], errors='coerce').notnull()]

# Step 2: Now safely convert to integer
df['label_id'] = df['Sentiment Score'].astype(int)

In [ ]:
def preprocess_tweet(tweet):
    if isinstance(tweet, str):
        tweet_words = []
        for word in tweet.split():
            if word.startswith('@'):
                word = '@user'
            elif word.startswith('http'):
                word = 'http'
            tweet_words.append(word)
        return " ".join(tweet_words)
    return ""

df['cleaned_text'] = df['text'].apply(preprocess_tweet)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label_id']
)

print(f"Training: {len(train_df)} samples")
print(f"Validation: {len(val_df)} samples")

Training: 5787 samples
Validation: 1447 samples


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
train_dataset = TweetDataset(
    train_df['cleaned_text'].tolist(),
    train_df['label_id'].tolist(),
    tokenizer
)

val_dataset = TweetDataset(
    val_df['cleaned_text'].tolist(),
    val_df['label_id'].tolist(),
    tokenizer
)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
import torch
from torch.optim import AdamW
from transformers import get_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

Using device: cpu


In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\nEpoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    accuracy = correct / total
    print(f"Epoch {epoch+1} - Validation Loss: {val_loss / len(val_loader):.4f} - Accuracy: {accuracy:.4f}")
    model.train()

  0%|          | 0/1086 [00:00<?, ?it/s]

IndexError: Target -1 is out of bounds.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from tqdm.auto import tqdm

# 1. Load data
df = pd.read_csv("/content/Final_Tweet_Data.csv")  # update if needed

# 2. Preprocess text
def preprocess_tweet(tweet):
    if isinstance(tweet, str):
        tweet_words = []
        for word in tweet.split():
            if word.startswith('@'):
                word = '@user'
            elif word.startswith('http'):
                word = 'http'
            tweet_words.append(word)
        return " ".join(tweet_words)
    return ""

df['cleaned_text'] = df['text'].apply(preprocess_tweet)

# 3. Clean labels
df = df[df['Sentiment Score'].isin([-1, 0, 1])]
label_map = {-1: 0, 0: 1, 1: 2}  # Map -1 → 0 (Negative), 0 → 1 (Neutral), 1 → 2 (Positive)
df['label_id'] = df['Sentiment Score'].map(label_map)

# 4. Train/test split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=42)

# 5. Load tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 6. Custom dataset class
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 7. Create datasets and loaders
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 8. Set device and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# 9. Set scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# 10. Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\nEpoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"Epoch {epoch+1} - Val Loss: {val_loss / len(val_loader):.4f} - Accuracy: {val_accuracy:.4f}")
    model.train()

# 11. Save fine-tuned model
model.save_pretrained("finetuned_tweet_sentiment_model")
tokenizer.save_pretrained("finetuned_tweet_sentiment_model")
print("✅ Model saved successfully.")

/tmp/ipython-input-27-3025986100.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['Sentiment Score'].map(label_map)


  0%|          | 0/1086 [00:00<?, ?it/s]

# RoBERTa-base (from Facebook)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Final_Tweet_Data.csv to Final_Tweet_Data.csv


In [ ]:
import pandas as pd

df = pd.read_csv('Final_Tweet_Data.csv')  # use the exact filename after upload
df.head()

,Brand,Factor,id,url,text,Sentiment Score,Tweet Type
0,Citroen,Price,1924160628727447785,https://x.com/TorqueIndia/status/1924160628727...,"""@MenonNinad Most of the cars comes with simil...",1.0,Consumer
1,Citroen,Price,1919053455513235793,https://x.com/grok/status/1919053455513235793,"""The Leapmotor T03 hatchback is expected to la...",1.0,Consumer
2,Citroen,Price,1919046080164958454,https://x.com/Xroaders_001/status/191904608016...,"""📣 Stellantis-Group a lease of new life or Las...",1.0,Consumer
3,Citroen,Price,1850832200490426383,https://x.com/RBlyskye7/status/185083220049042...,"""But unfortunately Citroën and Renault in Indo...",1.0,Consumer
4,Citroen,Price,1830534864191271150,https://x.com/TorqueIndia/status/1830534864191...,"""Citroen BASALT turbo affordable variants laun...",0.0,Consumer


In [ ]:
# 1. Install libraries if not already
!pip install transformers datasets --quiet

# 2. Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# 3. Load the CSV
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])  # drop empty rows

# 4. Clean the text
import re
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# 5. Drop invalid sentiment scores
df = df[df['Sentiment Score'] != -11.0]

# 6. Label mapping
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# 7. Train-test split
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label_id'])

# 8. Load tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 9. Custom Dataset class
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 10. Prepare datasets
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 11. Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"✅ Model ready on: {device}")

/tmp/ipython-input-3-504447856.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Model ready on: cuda


In [ ]:
num_epochs = 4  # You can change this to 4 or 5 if needed
num_training_steps = num_epochs * len(train_loader)

from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()

  0%|          | 0/1268 [00:00<?, ?it/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Number of epochs
num_epochs = 4

# Total training steps
num_training_steps = num_epochs * len(train_loader)

# Learning rate scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

  0%|          | 0/1268 [00:00<?, ?it/s]


✅ Epoch 1 - Training Loss: 0.7804
📊 Epoch 1 – Val Loss: 0.7184 – Accuracy: 0.6854

✅ Epoch 2 - Training Loss: 0.5647
📊 Epoch 2 – Val Loss: 0.7988 – Accuracy: 0.6836

✅ Epoch 3 - Training Loss: 0.3461
📊 Epoch 3 – Val Loss: 0.8479 – Accuracy: 0.7001

✅ Epoch 4 - Training Loss: 0.1762
📊 Epoch 4 – Val Loss: 1.1034 – Accuracy: 0.6863


# distilbert-base-uncased

In [ ]:
# 1. Install libraries if not already
!pip install transformers datasets --quiet

# 2. Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import re

# 3. Load the CSV
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])  # Drop missing rows

# 4. Clean the text
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# 5. Drop invalid sentiment score
df = df[df['Sentiment Score'] != -11.0]

# 6. Map labels
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# 7. Train-test split
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label_id'])

# 8. Load tokenizer and model (distilbert)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 9. Dataset class
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 10. Load data
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 11. Setup device and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 4
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# 12. Train
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Eval
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

# 13. Save model
model.save_pretrained("distilbert_sentiment_model")
tokenizer.save_pretrained("distilbert_sentiment_model")


/tmp/ipython-input-8-2308937856.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1268 [00:00<?, ?it/s]


✅ Epoch 1 - Training Loss: 0.8178
📊 Epoch 1 – Val Loss: 0.7920 – Accuracy: 0.6582

✅ Epoch 2 - Training Loss: 0.5711
📊 Epoch 2 – Val Loss: 0.7346 – Accuracy: 0.6965

✅ Epoch 3 - Training Loss: 0.3437
📊 Epoch 3 – Val Loss: 0.8390 – Accuracy: 0.6826

✅ Epoch 4 - Training Loss: 0.1758
📊 Epoch 4 – Val Loss: 1.0033 – Accuracy: 0.6895


('distilbert_sentiment_model/tokenizer_config.json',
 'distilbert_sentiment_model/special_tokens_map.json',
 'distilbert_sentiment_model/vocab.txt',
 'distilbert_sentiment_model/added_tokens.json',
 'distilbert_sentiment_model/tokenizer.json')

# BERTweet

In [ ]:
# 1. Install transformers & sentencepiece (for BERTweet)
!pip install transformers datasets sentencepiece --quiet

# 2. Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import re

# 3. Load dataset
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])
df = df[df['Sentiment Score'] != -11.0]  # remove bad row

# 4. Clean text
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# 5. Label encoding
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# 6. Train-test split (70-30)
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label_id'])

# 7. Load BERTweet
model_name = "vinai/bertweet-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 8. Custom dataset
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'][0],
            'attention_mask': enc['attention_mask'][0],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 9. Dataloaders
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 10. Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 4
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 11. Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

# 12. Save model
model.save_pretrained("bertweet_sentiment_model")
tokenizer.save_pretrained("bertweet_sentiment_model")


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

  0%|          | 0/1268 [00:00<?, ?it/s]


✅ Epoch 1 - Training Loss: 0.8004
📊 Epoch 1 – Val Loss: 0.8070 – Accuracy: 0.6366

✅ Epoch 2 - Training Loss: 0.6152
📊 Epoch 2 – Val Loss: 0.7133 – Accuracy: 0.7011

✅ Epoch 3 - Training Loss: 0.4139
📊 Epoch 3 – Val Loss: 0.7919 – Accuracy: 0.7015

✅ Epoch 4 - Training Loss: 0.2550
📊 Epoch 4 – Val Loss: 0.8955 – Accuracy: 0.7047


('bertweet_sentiment_model/tokenizer_config.json',
 'bertweet_sentiment_model/special_tokens_map.json',
 'bertweet_sentiment_model/vocab.txt',
 'bertweet_sentiment_model/bpe.codes',
 'bertweet_sentiment_model/added_tokens.json')

# twitter-roberta-base-sentiment

In [ ]:
# ✅ 1. Install libraries
!pip install transformers datasets --quiet

# ✅ 2. Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm.auto import tqdm
import re

# ✅ 3. Load data
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])

# ✅ 4. Clean tweets
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# ✅ 5. Remove bad sentiment values
df = df[df['Sentiment Score'] != -11.0]

# ✅ 6. Map labels
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# ✅ 7. Split data 60/40
train_df, val_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df['label_id'])

# ✅ 8. Load tokenizer/model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# ✅ 9. Dataset class
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# ✅ 10. Prepare datasets & loaders
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# ✅ 11. Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ 12. Optimizer & scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# ✅ 13. Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # ✅ Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

# ✅ 14. Save model
model.save_pretrained("roberta_sentiment_model_60_40")
tokenizer.save_pretrained("roberta_sentiment_model_60_40")

/tmp/ipython-input-4-3070761434.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

  0%|          | 0/816 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]


✅ Epoch 1 - Training Loss: 0.7740
📊 Epoch 1 – Val Loss: 0.7222 – Accuracy: 0.6935

✅ Epoch 2 - Training Loss: 0.5590
📊 Epoch 2 – Val Loss: 0.7581 – Accuracy: 0.7018

✅ Epoch 3 - Training Loss: 0.3491
📊 Epoch 3 – Val Loss: 0.8122 – Accuracy: 0.7132


('roberta_sentiment_model_60_40/tokenizer_config.json',
 'roberta_sentiment_model_60_40/special_tokens_map.json',
 'roberta_sentiment_model_60_40/vocab.json',
 'roberta_sentiment_model_60_40/merges.txt',
 'roberta_sentiment_model_60_40/added_tokens.json',
 'roberta_sentiment_model_60_40/tokenizer.json')

# deberta-v3-base

In [ ]:
# 1. Install libraries
!pip install transformers datasets --quiet

# 2. Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import re

# 3. Load and preprocess dataset
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])
df = df[df['Sentiment Score'] != -11.0]

def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# Label map
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# 4. 60-40 train-val split
train_df, val_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df['label_id'])

# 5. Load tokenizer and model
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 6. Custom Dataset
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 7. Prepare DataLoaders
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 8. Setup for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 9. Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

# 10. Save model
model.save_pretrained("deberta_sentiment_model")
tokenizer.save_pretrained("deberta_sentiment_model")


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/816 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]


✅ Epoch 1 - Training Loss: 0.9282
📊 Epoch 1 – Val Loss: 0.8085 – Accuracy: 0.5815

✅ Epoch 2 - Training Loss: 0.7381
📊 Epoch 2 – Val Loss: 0.7841 – Accuracy: 0.6348

✅ Epoch 3 - Training Loss: 0.6152
📊 Epoch 3 – Val Loss: 0.8054 – Accuracy: 0.6686


('deberta_sentiment_model/tokenizer_config.json',
 'deberta_sentiment_model/special_tokens_map.json',
 'deberta_sentiment_model/spm.model',
 'deberta_sentiment_model/added_tokens.json',
 'deberta_sentiment_model/tokenizer.json')

# bert-base-uncased

In [ ]:
# 1. Install & Import Libraries
!pip install transformers datasets --quiet

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import re

# 2. Load & Clean Data
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])

def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#', '', text)  # ✅ Fixed here
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)
df = df[df['Sentiment Score'] != -11.0]  # remove invalid
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# 3. 70-30 Train-Test Split
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label_id'])

# 4. Load Tokenizer and Model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 5. Custom Dataset
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 6. Prepare Data Loaders
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 7. Train Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 8. Training Loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

# 9. Save model
model.save_pretrained("bert_sentiment_model_70_30")
tokenizer.save_pretrained("bert_sentiment_model_70_30")

/tmp/ipython-input-13-3033547548.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/951 [00:00<?, ?it/s]


✅ Epoch 1 - Training Loss: 0.8132
📊 Epoch 1 – Val Loss: 0.7259 – Accuracy: 0.6909

✅ Epoch 2 - Training Loss: 0.5514
📊 Epoch 2 – Val Loss: 0.7485 – Accuracy: 0.6923

✅ Epoch 3 - Training Loss: 0.2875
📊 Epoch 3 – Val Loss: 0.8890 – Accuracy: 0.6969


('bert_sentiment_model_70_30/tokenizer_config.json',
 'bert_sentiment_model_70_30/special_tokens_map.json',
 'bert_sentiment_model_70_30/vocab.txt',
 'bert_sentiment_model_70_30/added_tokens.json',
 'bert_sentiment_model_70_30/tokenizer.json')

Doing a 70-30 model now

In [ ]:
print(df['label_id'].value_counts())

label_id
-1     3054
 0     2139
 1     2041
-11       1
Name: count, dtype: int64


# Final Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Save the model
model_dir = "final_roberta_sentiment_model"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

NameError: name 'model' is not defined

In [ ]:
def clean_tweet(text):
    import re
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

test_texts = [
    "This electric vehicle is amazing! I love how quiet it is.",
    "Battery died in the middle of nowhere. Ugh.",
    "Not bad, but still prefer petrol for longer drives."
]

for text in test_texts:
    cleaned = clean_tweet(text)
    inputs = tokenizer(cleaned, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=-1).item()
    confidence = probs[0][pred].item()

    print(f"📝 Text: {text}")
    print(f"🔮 Prediction: {label_map[pred]} ({confidence:.2f})")
    print("-" * 50)

📝 Text: This electric vehicle is amazing! I love how quiet it is.
🔮 Prediction: Positive (0.50)
--------------------------------------------------
📝 Text: Battery died in the middle of nowhere. Ugh.
🔮 Prediction: Negative (0.93)
--------------------------------------------------
📝 Text: Not bad, but still prefer petrol for longer drives.
🔮 Prediction: Positive (0.68)
--------------------------------------------------


In [ ]:
train_losses = []
val_losses = []
val_accuracies = []

# Inside training loop
train_losses.append(total_loss / len(train_loader))
val_losses.append(val_loss / len(val_loader))
val_accuracies.append(val_accuracy)

In [ ]:
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch

# Logging lists
train_losses = []
val_losses = []
val_accuracies = []

# Training settings
num_epochs = 4
num_training_steps = num_epochs * len(train_loader)
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        progress_bar.update(1)

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    print(f"\n✅ Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)

    print(f"📊 Epoch {epoch+1} – Val Loss: {avg_val_loss:.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()


  0%|          | 0/1268 [00:00<?, ?it/s]


✅ Epoch 1 - Training Loss: 0.1959
📊 Epoch 1 – Val Loss: 0.8890 – Accuracy: 0.6969

✅ Epoch 2 - Training Loss: 0.1964
📊 Epoch 2 – Val Loss: 0.8890 – Accuracy: 0.6969

✅ Epoch 3 - Training Loss: 0.1948
📊 Epoch 3 – Val Loss: 0.8890 – Accuracy: 0.6969

✅ Epoch 4 - Training Loss: 0.1944
📊 Epoch 4 – Val Loss: 0.8890 – Accuracy: 0.6969


# **Roberta 75-25 Final Model**

In [ ]:
# ✅ 1. Install libraries
!pip install transformers datasets --quiet

# ✅ 2. Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm.auto import tqdm
import re

# ✅ 3. Load data
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])

# ✅ 4. Clean tweets
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# ✅ 5. Remove bad sentiment values
df = df[df['Sentiment Score'] != -11.0]

# ✅ 6. Map labels
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)

# ✅ 7. Split data 75/25
train_df, val_df = train_test_split(df, test_size=0.25, random_state=42, stratify=df['label_id'])

# ✅ 8. Load tokenizer/model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# ✅ 9. Dataset class
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# ✅ 10. Prepare datasets & loaders
train_dataset = TweetDataset(train_df['cleaned_text'].tolist(), train_df['label_id'].tolist(), tokenizer)
val_dataset = TweetDataset(val_df['cleaned_text'].tolist(), val_df['label_id'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# ✅ 11. Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ 12. Optimizer & scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# ✅ 13. Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        total_loss += loss.item()

    print(f"\n✅ Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

    # ✅ Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_accuracy = correct / total
    print(f"📊 Epoch {epoch+1} – Val Loss: {val_loss / len(val_loader):.4f} – Accuracy: {val_accuracy:.4f}")
    model.train()

# ✅ 14. Save model
model.save_pretrained("roberta_sentiment_model_60_40")
tokenizer.save_pretrained("roberta_sentiment_model_60_40")

/tmp/ipython-input-5-4024380487.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['Sentiment Score'].map(label_map).astype(int)


  0%|          | 0/1020 [00:00<?, ?it/s]


✅ Epoch 1 - Training Loss: 0.7711
📊 Epoch 1 – Val Loss: 0.7131 – Accuracy: 0.6794

✅ Epoch 2 - Training Loss: 0.5526
📊 Epoch 2 – Val Loss: 0.6618 – Accuracy: 0.7197

✅ Epoch 3 - Training Loss: 0.3399
📊 Epoch 3 – Val Loss: 0.7962 – Accuracy: 0.7192


('roberta_sentiment_model_60_40/tokenizer_config.json',
 'roberta_sentiment_model_60_40/special_tokens_map.json',
 'roberta_sentiment_model_60_40/vocab.json',
 'roberta_sentiment_model_60_40/merges.txt',
 'roberta_sentiment_model_60_40/added_tokens.json',
 'roberta_sentiment_model_60_40/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from tqdm.auto import tqdm

# Load your fine-tuned model
model_path = "roberta_sentiment_model_60_40"  # path to your saved model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Updated label mapping
label_map = {0: -1, 1: 0, 2: 1}  # Model class → Sentiment Score
label_name_map = {-1: 'Negative', 0: 'Neutral', 1: 'Positive'}

# Function to predict sentiment with correct mapping
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted = torch.argmax(probs, dim=-1).item()
    sentiment_score = label_map[predicted]
    sentiment_label = label_name_map[sentiment_score]
    return sentiment_label

# Add predicted sentiment column
tqdm.pandas()
df['Predicted Sentiment'] = df['cleaned_text'].progress_apply(predict_sentiment)

# Group and pivot
sentiment_counts = df.groupby(['Brand', 'Factor', 'Predicted Sentiment']).size().reset_index(name='Count')
sentiment_pivot = sentiment_counts.pivot_table(index=['Brand', 'Factor'], columns='Predicted Sentiment', values='Count', fill_value=0).reset_index()

# Optional score: Positive - Negative
def sentiment_score(row):
    return row.get('Positive', 0) - row.get('Negative', 0)

sentiment_pivot['Sentiment Score'] = sentiment_pivot.apply(sentiment_score, axis=1)

  0%|          | 0/7234 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

# Step 1: Load your original data
df = pd.read_csv("/content/Final_Tweet_Data.csv")
df = df.dropna(subset=["text", "Brand", "Factor"])

# Step 2: Clean text
def clean_text(text):
    import re
    text = re.sub(r"http\S+|www\S+|https\S+", '', str(text))
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

df["cleaned_text"] = df["text"].apply(clean_text)

# Step 3: Load your fine-tuned RoBERTa model
model_path = "roberta_sentiment_model_60_40"  # replace with correct path if needed
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# Step 4: Dataset for inference
class TweetDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
        }

# Step 5: Predict sentiments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = TweetDataset(df["cleaned_text"].tolist(), tokenizer)
loader = DataLoader(dataset, batch_size=32)

predictions = []
model.eval()
with torch.no_grad():
    for batch in tqdm(loader, desc="Predicting"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())

# Step 6: Map predictions to labels
reverse_label_map = {0: -1, 1: 0, 2: 1}
df["predicted_label"] = [reverse_label_map[p] for p in predictions]
df["Sentiment"] = df["predicted_label"].map({-1: "Negative", 0: "Neutral", 1: "Positive"})

# Step 7: Group and calculate sentiment scores
summary = df.groupby(["Brand", "Factor", "Sentiment"]).size().unstack(fill_value=0)
summary["Total"] = summary.sum(axis=1)

for sentiment in ["Positive", "Neutral", "Negative"]:
    if sentiment not in summary.columns:
        summary[sentiment] = 0
    summary[f"{sentiment} %"] = (summary[sentiment] / summary["Total"] * 100).round(1).astype(str) + "%"

# Step 8: Format and show result
summary = summary.reset_index()
pd.set_option("display.max_rows", 100)
print(summary)


Predicting:   0%|          | 0/227 [00:00<?, ?it/s]

Sentiment          Brand       Factor  Negative  Neutral  Positive  Total  \
0                      1        Price         0        0         1      1   
1                Citroen  After Sales        82       18        16    116   
2                Citroen        Brand        48       34        41    123   
3                Citroen      Finance         0        2        13     15   
4                Citroen        Price         2       10        45     57   
5                Citroen       Safety        45       17        23     85   
6                  Honda   AfterSales       140       17        17    174   
7                  Honda       Brand         59       32        52    143   
8                  Honda      Finance         1        3        19     23   
9                  Honda        Price        31       25        24     80   
10                 Honda       Safety        39       19        59    117   
11               Hyundai   AfterSales        98       19        40    157   

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re
from IPython.display import display

# Load the data
df = pd.read_csv('/content/Final_Tweet_Data.csv')
df = df.dropna(subset=['text', 'Sentiment Score'])

# Remove invalid brand names (like numeric "1")
df = df[~df['Brand'].astype(str).str.isnumeric()]

# Clean the text
def clean_tweet(text):
    text = str(text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['text'].apply(clean_tweet)

# Load tokenizer and model
model_path = "roberta_sentiment_model_60_40"  # Update if your model path is different
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Custom Dataset for inference
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0]
        }

# Prepare DataLoader
inference_dataset = InferenceDataset(df['cleaned_text'].tolist(), tokenizer)
inference_loader = DataLoader(inference_dataset, batch_size=32)

# Predict
all_preds = []
with torch.no_grad():
    for batch in inference_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())

# Map predictions to sentiment labels
reverse_label_map = {0: -1, 1: 0, 2: 1}
df['predicted_label'] = [reverse_label_map[pred] for pred in all_preds]

# Group by Brand and Factor
summary = df.groupby(['Brand', 'Factor', 'predicted_label']).size().unstack(fill_value=0).reset_index()
summary.columns.name = None

# Ensure all sentiment columns exist
for col in [-1, 0, 1]:
    if col not in summary.columns:
        summary[col] = 0

# Rename sentiment columns
summary = summary.rename(columns={
    -1: 'Negative',
    0: 'Neutral',
    1: 'Positive'
})

# Calculate totals and percentages
summary['Total'] = summary[['Positive', 'Neutral', 'Negative']].sum(axis=1)
summary['Positive %'] = (summary['Positive'] / summary['Total'] * 100).round(1).astype(str) + '%'
summary['Neutral %'] = (summary['Neutral'] / summary['Total'] * 100).round(1).astype(str) + '%'
summary['Negative %'] = (summary['Negative'] / summary['Total'] * 100).round(1).astype(str) + '%'

# Reorder columns
final_summary = summary[['Brand', 'Factor', 'Positive', 'Neutral', 'Negative', 'Total',
                         'Positive %', 'Neutral %', 'Negative %']]

# Display results
display(final_summary)

,Brand,Factor,Positive,Neutral,Negative,Total,Positive %,Neutral %,Negative %
0,Citroen,After Sales,16,18,82,116,13.8%,15.5%,70.7%
1,Citroen,Brand,41,34,48,123,33.3%,27.6%,39.0%
2,Citroen,Finance,13,2,0,15,86.7%,13.3%,0.0%
3,Citroen,Price,44,9,2,55,80.0%,16.4%,3.6%
4,Citroen,Safety,23,17,45,85,27.1%,20.0%,52.9%
5,Honda,AfterSales,17,17,140,174,9.8%,9.8%,80.5%
6,Honda,Brand,52,32,59,143,36.4%,22.4%,41.3%
7,Honda,Finance,19,3,1,23,82.6%,13.0%,4.3%
8,Honda,Price,24,25,31,80,30.0%,31.2%,38.8%
9,Honda,Safety,59,19,39,117,50.4%,16.2%,33.3%
